In [3]:
# 環境変数取得
ENV["APPDATA"]

"C:\\Users\\user\\AppData\\Roaming"

In [2]:
"""
Selenium WebDriver 動作確認
"""

using PyCall

# python/selenium パッケージ import
const webdriver = pyimport("selenium.webdriver")
const WebDriverWait = pyimport("selenium.webdriver.support.ui").WebDriverWait
const EC = pyimport("selenium.webdriver.support.expected_conditions")
# python/chromedriver_binary パッケージ import
pyimport("chromedriver_binary")

# ChromeWebDriverで Google WEBサイト取得
driver = webdriver.Chrome()
driver.get("https://www.google.co.jp")
# ロード待ち
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located)
# 検索窓に "ChromeDriver" を入力し submit
search_box = driver.find_element_by_name("q")
search_box.send_keys("ChromeDriver")
search_box.submit()
# ロード待ち
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located)
# スクリーンショット保存
driver.save_screenshot("screenshot.png")
# ChromeWebDriver終了
driver.quit()

In [5]:
# さくらレンタルサーバ｜ログイン
driver = webdriver.Chrome()
driver.get("https://secure.sakura.ad.jp/rscontrol")
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located)
driver.find_element_by_xpath("//input[@name=\"domain\"]").send_keys(ENV["SAKURA_USER"])
driver.find_element_by_xpath("//input[@name=\"password\"]").send_keys(ENV["SAKURA_PASSWD"])
driver.find_element_by_xpath("//input[@class=\"image\"]").submit()
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located)
println(driver.current_url)
driver.quit()

https://secure.sakura.ad.jp/rscontrol/main/


In [19]:
isnothing(match(r"^https://secure\.sakura\.ad\.jp/rscontrol/", "https://secure.sakura.ad.jp/rscontrol/main/"))

false

In [20]:
# Selenium制御ブロック
selenium(callback) = begin
    driver = webdriver.Chrome()
    callback(driver)
    driver.quit()
end

# さくらレンタルサーバーログイン
sakura_login(driver, login_user, login_passwd) = begin
    driver.get("https://secure.sakura.ad.jp/rscontrol")
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located)
    driver.find_element_by_xpath("//input[@name=\"domain\"]").send_keys(login_user)
    driver.find_element_by_xpath("//input[@name=\"password\"]").send_keys(login_passwd)
    driver.find_element_by_xpath("//input[@class=\"image\"]").submit()
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located)
    return driver.current_url
end

# 無料SSL証明書（Let's Encrypt）申し込み
sakura_apply_lets_encrypt(driver, panel_url, domain) = begin
    driver.get("$(panel_url)freessl?SNIDomain=$(domain)")
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located)
    buttons = driver.find_elements_by_xpath("//button[@type=\"submit\"]")
    length(buttons) === 0 && return false
    buttons[0].submit()
    return true
end

# ログインして無料SSL申請
selenium() do driver
    panel_url = sakura_login(driver, ENV["SAKURA_USER"], ENV["SAKURA_PASSWD"])
    isnothing(match(r"^https://secure\.sakura\.ad\.jp/rscontrol/", panel_url)) && return false
    sakura_apply_lets_encrypt(driver, panel_url, "example.com")
end